In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile

import numpy as np
import pytorch_lightning as pl
import torch
import datamol as dm

import goli

Using backend: pytorch


## Goli data modules
    
- All the below config are primitive types and can be easily embeded in a YAML config file.
- Loading a datamodule should not require `config_load_dataset` and ideally would only require `goli.data.DGLFromSmilesDataModule(**dm_args)`.
- At the moment we only have a single datamodule but we'll likely have others in the future. In that case we could eventually have a `config_load_dataset` that just select the appropriate data module.

Here is a config (only for the data) that is similar to the current one in `goli/expts/config_micro_ZINC.yaml` but adjusted to be used directly with `DGLFromSmilesDataModule`:

```yaml
data:
  module_type: "DGLFromSmilesDataModule"
  args:
    df_path: null  # could be set from the CLI. Ideally config files does not have paths.
    cache_data_path: null  # could be set from the CLI. Ideally config files does not have paths.
  
    smiles_col: "SMILES"
    label_cols: ["SA"]
    split_val: 0.2
    split_test: 0.2
    split_seed: 19
  
    train_val_batch_size: 16
    test_batch_size: 16
  
    featurization:
      atom_property_list_float: []
      atom_property_list_onehot: ["atomic-number", "degree"]
      edge_property_list: ["ring", "bond-type-onehot"]
      add_self_loop: false
      use_bonds_weights: false
      explicit_H: false
```

---

In [3]:
# Setup a temporary cache file. Only for
# demo purposes, use a known path in prod.
cache_data_path = pathlib.Path(tempfile.mkdtemp()) / "cache.pkl"

# Load a dataframe
df = goli.data.load_tiny_zinc()
df.head()

# Setup the featurization
featurization_args = {}
featurization_args["atom_property_list_float"] = []  # ["weight", "valence"]
featurization_args["atom_property_list_onehot"] = ["atomic-number", "degree"]
featurization_args["edge_property_list"] = ["ring", "bond-type-onehot"]
featurization_args["add_self_loop"] = False
featurization_args["use_bonds_weights"] = False
featurization_args["explicit_H"] = False

# Config for datamodule
dm_args = {}
dm_args["df"] = df
dm_args["cache_data_path"] = None#cache_data_path  # unsed at the moment
dm_args["featurization"] = featurization_args
dm_args["smiles_col"] = "SMILES"
dm_args["label_cols"] = ["SA"]
dm_args["split_val"] = 0.2
dm_args["split_test"] = 0.2
dm_args["split_seed"] = 19
dm_args["train_val_batch_size"] = 16
dm_args["test_batch_size"] = 16
dm_args["num_workers"] = 0
dm_args["pin_memory"] = True
dm_args["featurization_n_jobs"] = 16
dm_args["featurization_progress"] = True


dm = goli.data.DGLFromSmilesDataModule(**dm_args)
dm

In [4]:
# Load and prepare the data
dm.prepare_data()

# Create the split torch datasets
dm.setup()

2021-03-16 18:39:34.142 | INFO     | goli.data.datamodule:prepare_data:171 - Prepare dataset with 100 data points.


  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
# Load a dataloader and get the first batch from it
dl = dm.train_dataloader()
it = iter(dl)
batch = next(it)
batch

{'smiles': ['c1cc2c(cc1N[C@@H]1CCOC3(CCC3)C1)CCC2',
  'CC(=O)N1CC[C@@H]([NH2+][C@@H](C)CSCC(C)C)C1',
  'Cc1ccc(-c2nc3ccc(C)c(C)c3[nH]2)nc1',
  'CCOC[C@H](O)[C@](C)(CC)[NH+]1CCCC1',
  'Cc1cc(CC(=O)N[C@H](c2ccc(F)cc2)C2CCC2)no1',
  'CCc1ccc(C(=O)/C(=C(/S)NC2CC2)[n+]2ccc(CC)cc2)cc1',
  'Cc1ccc(NC(=O)c2ccc(F)cc2F)cc1S(=O)(=O)Nc1ccc(Cl)cc1',
  'CC#CCCC(=O)Nc1cccc2c1C(=O)c1ccccc1C2=O',
  'CCOc1cc(/C=C(\\C#N)C(=O)c2c[nH]c3cc(Cl)ccc23)ccc1OC',
  'CNC(=O)[C@@H]1CCC[NH+]1Cc1ccc(C)c(F)c1',
  'CC(C)(C)OC(=O)N[C@H]1CCN(c2cc(-c3cccs3)n[nH]2)C1',
  'Cc1nn(-c2ccccc2)c(O)c1/C=[NH+]/Cc1ccncc1',
  'COCC[NH+](C)Cc1c(C)cc(C)c(C(C)=O)c1C',
  'Cc1nc(C)c(S(=O)(=O)/N=C(\\[O-])C[C@H]2CCCO2)s1',
  'COc1cccc(CN2CCC[NH+](CC(=O)Nc3ccc(F)cc3)S2(=O)=O)c1',
  'COc1cc(F)cc(CNC(=O)[C@H]2CCCN2C(=O)Cc2ccccc2)c1'],
 'features': Graph(num_nodes=352, num_edges=754,
       ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}),
 'labels': tensor

---

## Launch a training

In `goli.cli.train` I have added a `click` CLI command that take a config file as input and build the datamodule. Once I am done with the PL module I will complete the command.

The way to use it is quite simple, you need to install goli with `pip install -e .` (omit`-e` in prod) and then:

```bash
goli train -c my_config.yaml
```

It's not here for now but usually I "augment" the CLI command with various config key you might want to set without having to modify the config file itself:

```bash
goli train -c my_config.yaml --training-path /home/hadim/data/goli/runs/exp_1
```

Later the same strategy could be done to launch an hparams tuning run (with a config file as above + a config file defning the search space).

```bash
goli tune -c my_config.yaml --tune-config tuning_space.yaml
```